## Feature Engineering to Calculate Interest Rate
This notebook creates the base dataset to calculate the meta-feature "CNT_PAYMENT", which is the term of previous credit of a previous application at Home Credit. This feature is part of the previous application dataset ("previous_application.csv') but not part of the current application dataset ('application_test.csv' and 'application_train.csv'), and we need this number to calculate the INTEREST, which is a very important indicator of credit rating.

In [1]:
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
pd.options.mode.chained_assignment = None  # default='warn'

/opt/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


### Reading in datasets and calculating preliminary ratios
AMT_CREDIT is the amount of credit actually given to the customer after application

In [2]:
t2 = pd.read_csv('application_test.csv')
t1 = pd.read_csv('application_train.csv')
tr = pd.read_csv('previous_application.csv')
tr=tr.dropna(axis=0,subset=['CNT_PAYMENT'])
tr['CNT_PAYMENT'] = tr['CNT_PAYMENT'].astype('int')

for i in [t1, t2, tr]:
    i['diff/goods']=(i.AMT_CREDIT-i.AMT_GOODS_PRICE)/i.AMT_GOODS_PRICE 
    i['diff/credit']=(i.AMT_CREDIT-i.AMT_GOODS_PRICE)/i.AMT_CREDIT
    i['credit_to_annuity']=i.AMT_CREDIT/i.AMT_ANNUITY
    i['price_to_annuity']=i.AMT_GOODS_PRICE/i.AMT_ANNUITY
    i['simple_diff']=i.AMT_CREDIT-i.AMT_GOODS_PRICE
    i['credit_term']=i.AMT_ANNUITY/i.AMT_CREDIT


In [3]:
tr.shape

(1297984, 43)

CNT_PAYMENT is the target variable

In [4]:
tr['CNT_PAYMENT'].isna().sum()
cnt=tr['CNT_PAYMENT']

In [5]:
tr.head(10)

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,diff/goods,diff/credit,credit_to_annuity,price_to_annuity,simple_diff,credit_term
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,300.0,-42.0,-37.0,0.0,0.000000,0.000000,9.907942,9.907942,0.0,0.100929
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,916.0,365243.0,365243.0,1.0,0.118800,0.106185,26.983262,24.118039,72171.0,0.037060
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,59.0,365243.0,365243.0,1.0,0.212840,0.175489,9.059618,7.469755,23944.5,0.110380
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,-152.0,-182.0,-177.0,1.0,0.046200,0.044160,10.008007,9.566055,20790.0,0.099920
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,NaN,NaN,NaN,NaN,0.197200,0.164718,12.656622,10.571853,66555.0,0.079010
5,1383531,199383,Cash loans,23703.930,315000.0,340573.5,NaN,315000.0,SATURDAY,8,...,-144.0,-144.0,-137.0,1.0,0.081186,0.075090,14.367807,13.288936,25573.5,0.069600
10,1715995,447712,Cash loans,11368.620,270000.0,335754.0,NaN,270000.0,FRIDAY,7,...,885.0,-345.0,-334.0,1.0,0.243533,0.195840,29.533400,23.749584,65754.0,0.033860
11,2257824,161140,Cash loans,13832.775,211500.0,246397.5,NaN,211500.0,FRIDAY,10,...,85.0,-725.0,-721.0,1.0,0.165000,0.141631,17.812586,15.289774,34897.5,0.056140
12,2330894,258628,Cash loans,12165.210,148500.0,174361.5,NaN,148500.0,TUESDAY,15,...,-140.0,-200.0,-197.0,1.0,0.174152,0.148321,14.332798,12.206941,25861.5,0.069770
13,1397919,321676,Consumer loans,7654.860,53779.5,57564.0,0.0,53779.5,SUNDAY,15,...,-168.0,-168.0,-163.0,1.0,0.070371,0.065744,7.519929,7.025537,3784.5,0.132980


In [6]:
df2=t1[tr.columns & t1.columns]
df2=pd.DataFrame(df2)
names=df2.columns
tr=tr[names]
t1=t1[names]
t2=t2[names]

In [7]:
tr['CNT_PAYMENT']=cnt
tr['CNT_PAYMENT'].isna().sum()

0

### Numeric aggregations and merging of datasets

In [8]:
#aggregate funcs for numeric cols
def agg_numeric(df, group_var, df_name):
    """Aggregates the numeric values in a dataframe. This can
    be used to create features for each instance of the grouping variable.
    
    Parameters
    --------
        df (dataframe): 
            the dataframe to calculate the statistics on
        group_var (string): 
            the variable by which to group df
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated for 
            all numeric columns. Each instance of the grouping variable will have the statistics (mean, min, max, sum; currently supported) calculated. 
            The columns are also renamed to keep track of features created.
    
    """
    # Remove id variables other than grouping variable
    #print(df.head)
    for col in df:
        if col != group_var and 'SK_ID_CURR' and 'CNT_PAYMENT' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids
    #print(numeric_df[numeric_df['SK_ID_CURR']==108129])
    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()
    #print(agg.head())
    
    # Need to create new column names
    columns = [group_var]

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    return agg


def count_categorical(df, group_var, df_name):
# Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    categorical.head()
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

In [9]:
tr_agg = agg_numeric(tr, group_var = 'SK_ID_CURR', df_name = 'tr')
tr_agg.head(10)

,SK_ID_CURR,tr_AMT_ANNUITY_count,tr_AMT_ANNUITY_mean,tr_AMT_ANNUITY_max,tr_AMT_ANNUITY_min,tr_AMT_ANNUITY_sum,tr_AMT_CREDIT_count,tr_AMT_CREDIT_mean,tr_AMT_CREDIT_max,tr_AMT_CREDIT_min,...,tr_simple_diff_count,tr_simple_diff_mean,tr_simple_diff_max,tr_simple_diff_min,tr_simple_diff_sum,tr_credit_term_count,tr_credit_term_mean,tr_credit_term_max,tr_credit_term_min,tr_credit_term_sum
0,100001,1,3951.000000,3951.000,3951.000,3951.000,1,23787.000000,23787.0,23787.0,...,1,-1048.500000,-1048.5,-1048.5,-1048.50,1,0.166099,0.166099,0.166099,0.166099
1,100002,1,9251.775000,9251.775,9251.775,9251.775,1,179055.000000,179055.0,179055.0,...,1,0.000000,0.0,0.0,0.00,1,0.051670,0.051670,0.051670,0.051670
2,100003,3,56553.990000,98356.995,6737.310,169661.970,3,484191.000000,1035882.0,68053.5,...,3,48754.500000,135882.0,-756.0,146263.50,3,0.126383,0.185200,0.094950,0.379150
3,100004,1,5357.250000,5357.250,5357.250,5357.250,1,20106.000000,20106.0,20106.0,...,1,-4176.000000,-4176.0,-4176.0,-4176.00,1,0.266450,0.266450,0.266450,0.266450
4,100005,1,4813.200000,4813.200,4813.200,4813.200,1,40153.500000,40153.5,40153.5,...,1,-4464.000000,-4464.0,-4464.0,-4464.00,1,0.119870,0.119870,0.119870,0.119870
5,100006,6,23651.175000,39954.510,2482.920,141907.050,6,437543.250000,906615.0,24219.0,...,6,29238.360000,218115.0,-66987.0,175430.16,6,0.069304,0.108340,0.035920,0.415823
6,100007,6,12278.805000,22678.785,1834.290,73672.830,6,166638.750000,284400.0,14616.0,...,6,16108.500000,59400.0,-2560.5,96651.00,6,0.090659,0.125499,0.045749,0.543953
7,100008,4,15839.696250,25309.575,8019.090,63358.785,4,203459.625000,501975.0,39955.5,...,4,8832.375000,51975.0,-12145.5,35329.50,4,0.118055,0.200701,0.050420,0.472221
8,100009,7,10051.412143,17341.605,7435.845,70359.885,7,70137.642857,98239.5,38574.0,...,7,-6604.071429,0.0,-17671.5,-46228.50,7,0.155250,0.197370,0.091580,1.086750
9,100010,1,27463.410000,27463.410,27463.410,27463.410,1,260811.000000,260811.0,260811.0,...,1,13599.000000,13599.0,13599.0,13599.00,1,0.105300,0.105300,0.105300,0.105300


In [10]:
tr_count = count_categorical(tr, group_var = 'SK_ID_CURR', df_name = 'tr')
del tr_count['tr_NAME_CONTRACT_TYPE_Consumer loans_count']
del tr_count['tr_NAME_CONTRACT_TYPE_Consumer loans_count_norm']
tr_count.head()

,tr_NAME_CONTRACT_TYPE_Cash loans_count,tr_NAME_CONTRACT_TYPE_Cash loans_count_norm,tr_NAME_CONTRACT_TYPE_Revolving loans_count,tr_NAME_CONTRACT_TYPE_Revolving loans_count_norm,tr_WEEKDAY_APPR_PROCESS_START_FRIDAY_count,tr_WEEKDAY_APPR_PROCESS_START_FRIDAY_count_norm,tr_WEEKDAY_APPR_PROCESS_START_MONDAY_count,tr_WEEKDAY_APPR_PROCESS_START_MONDAY_count_norm,tr_WEEKDAY_APPR_PROCESS_START_SATURDAY_count,tr_WEEKDAY_APPR_PROCESS_START_SATURDAY_count_norm,...,tr_NAME_TYPE_SUITE_Group of people_count,tr_NAME_TYPE_SUITE_Group of people_count_norm,tr_NAME_TYPE_SUITE_Other_A_count,tr_NAME_TYPE_SUITE_Other_A_count_norm,tr_NAME_TYPE_SUITE_Other_B_count,tr_NAME_TYPE_SUITE_Other_B_count_norm,"tr_NAME_TYPE_SUITE_Spouse, partner_count","tr_NAME_TYPE_SUITE_Spouse, partner_count_norm",tr_NAME_TYPE_SUITE_Unaccompanied_count,tr_NAME_TYPE_SUITE_Unaccompanied_count_norm
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,0,0.000000,0,0.0,1,1.000000,0,0.0,0,0.000000,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000
100002,0,0.000000,0,0.0,0,0.000000,0,0.0,1,1.000000,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000
100003,1,0.333333,0,0.0,1,0.333333,0,0.0,1,0.333333,...,0,0.0,0,0.0,0,0.0,0,0.0,1,0.333333
100004,0,0.000000,0,0.0,1,1.000000,0,0.0,0,0.000000,...,0,0.0,0,0.0,0,0.0,0,0.0,1,1.000000
100005,0,0.000000,0,0.0,0,0.000000,0,0.0,0,0.000000,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000


In [11]:
tr=tr.merge(tr_agg,how='left',on='SK_ID_CURR')
tr.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,NAME_TYPE_SUITE,diff/goods,diff/credit,...,tr_simple_diff_count,tr_simple_diff_mean,tr_simple_diff_max,tr_simple_diff_min,tr_simple_diff_sum,tr_credit_term_count,tr_credit_term_mean,tr_credit_term_max,tr_credit_term_min,tr_credit_term_sum
0,271877,Consumer loans,1730.430,17145.0,17145.0,SATURDAY,15,NaN,0.00000,0.000000,...,3,-11277.000000,11448.0,-45279.00,-33831.00,3,0.081146,0.103609,0.03890,0.243438
1,108129,Cash loans,25188.615,679671.0,607500.0,THURSDAY,11,Unaccompanied,0.11880,0.106185,...,5,22195.800000,72171.0,-23562.00,110979.00,6,0.114463,0.260010,0.03706,0.686780
2,122040,Cash loans,15060.735,136444.5,112500.0,TUESDAY,11,"Spouse, partner",0.21284,0.175489,...,3,8008.500000,23944.5,-5715.00,24025.50,3,0.099027,0.116700,0.07000,0.297080
3,176158,Cash loans,47041.335,470790.0,450000.0,MONDAY,7,NaN,0.04620,0.044160,...,21,14099.550000,72931.5,-10525.95,296090.55,21,0.101329,0.203268,0.03809,2.127902
4,202054,Cash loans,31924.395,404055.0,337500.0,THURSDAY,9,NaN,0.19720,0.164718,...,17,42682.235294,126256.5,-2470.50,725598.00,17,0.097660,0.193271,0.05000,1.660217


In [12]:
tr=tr.merge(tr_count,how='left',on='SK_ID_CURR')
tr.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,NAME_TYPE_SUITE,diff/goods,diff/credit,...,tr_NAME_TYPE_SUITE_Group of people_count,tr_NAME_TYPE_SUITE_Group of people_count_norm,tr_NAME_TYPE_SUITE_Other_A_count,tr_NAME_TYPE_SUITE_Other_A_count_norm,tr_NAME_TYPE_SUITE_Other_B_count,tr_NAME_TYPE_SUITE_Other_B_count_norm,"tr_NAME_TYPE_SUITE_Spouse, partner_count","tr_NAME_TYPE_SUITE_Spouse, partner_count_norm",tr_NAME_TYPE_SUITE_Unaccompanied_count,tr_NAME_TYPE_SUITE_Unaccompanied_count_norm
0,271877,Consumer loans,1730.430,17145.0,17145.0,SATURDAY,15,NaN,0.00000,0.000000,...,0,0.0,0,0.0,0,0.0,0,0.000000,0,0.000000
1,108129,Cash loans,25188.615,679671.0,607500.0,THURSDAY,11,Unaccompanied,0.11880,0.106185,...,0,0.0,0,0.0,0,0.0,0,0.000000,3,0.500000
2,122040,Cash loans,15060.735,136444.5,112500.0,TUESDAY,11,"Spouse, partner",0.21284,0.175489,...,0,0.0,0,0.0,0,0.0,2,0.666667,0,0.000000
3,176158,Cash loans,47041.335,470790.0,450000.0,MONDAY,7,NaN,0.04620,0.044160,...,0,0.0,0,0.0,0,0.0,0,0.000000,6,0.285714
4,202054,Cash loans,31924.395,404055.0,337500.0,THURSDAY,9,NaN,0.19720,0.164718,...,0,0.0,0,0.0,0,0.0,2,0.117647,5,0.294118


In [13]:
t1_agg = agg_numeric(t1, group_var = 'SK_ID_CURR', df_name = 'tr')
t1_count = count_categorical(t1, group_var = 'SK_ID_CURR', df_name = 'tr')
t1=t1.merge(t1_agg,how='left',on='SK_ID_CURR')
t1=t1.merge(t1_count,how='left',on='SK_ID_CURR')
t1.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,NAME_TYPE_SUITE,diff/goods,diff/credit,...,tr_NAME_TYPE_SUITE_Group of people_count,tr_NAME_TYPE_SUITE_Group of people_count_norm,tr_NAME_TYPE_SUITE_Other_A_count,tr_NAME_TYPE_SUITE_Other_A_count_norm,tr_NAME_TYPE_SUITE_Other_B_count,tr_NAME_TYPE_SUITE_Other_B_count_norm,"tr_NAME_TYPE_SUITE_Spouse, partner_count","tr_NAME_TYPE_SUITE_Spouse, partner_count_norm",tr_NAME_TYPE_SUITE_Unaccompanied_count,tr_NAME_TYPE_SUITE_Unaccompanied_count_norm
0,100002,Cash loans,24700.5,406597.5,351000.0,WEDNESDAY,10,Unaccompanied,0.158397,0.136738,...,0,0,0,0,0,0,0,0,1,1
1,100003,Cash loans,35698.5,1293502.5,1129500.0,MONDAY,11,Family,0.145199,0.126789,...,0,0,0,0,0,0,0,0,0,0
2,100004,Revolving loans,6750.0,135000.0,135000.0,MONDAY,9,Unaccompanied,0.000000,0.000000,...,0,0,0,0,0,0,0,0,1,1
3,100006,Cash loans,29686.5,312682.5,297000.0,WEDNESDAY,17,Unaccompanied,0.052803,0.050155,...,0,0,0,0,0,0,0,0,1,1
4,100007,Cash loans,21865.5,513000.0,513000.0,THURSDAY,11,Unaccompanied,0.000000,0.000000,...,0,0,0,0,0,0,0,0,1,1


In [14]:
t2_agg = agg_numeric(t2, group_var = 'SK_ID_CURR', df_name = 'tr')
t2_count = count_categorical(t2, group_var = 'SK_ID_CURR', df_name = 'tr')
t2=t2.merge(t2_agg,how='left',on='SK_ID_CURR')
t2=t2.merge(t2_count,how='left',on='SK_ID_CURR')

### One-hot encoding of dummy variables

In [15]:
tr=pd.get_dummies(tr)
t1=pd.get_dummies(t1)
t2=pd.get_dummies(t2)

In [16]:
del tr['NAME_CONTRACT_TYPE_Consumer loans']

### Exporting to csvs

In [17]:
tr.to_csv(r'train.csv')
t1.to_csv(r'test1.csv')
t2.to_csv(r'test2.csv')